In [ ]:
#Import data

In [1]:
from __future__ import print_function  # Python 2 and 3
import pandas as pd
import scipy.stats
import numpy as np
from numpy.random import shuffle
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn import linear_model
from sklearn.linear_model import Ridge, Lasso, ElasticNet
import xgboost as xgb
from sklearn.svm import SVR
#from scipy import stats
import matplotlib.pyplot as plt
from scipy import stats
%matplotlib inline
plt.style.use('ggplot')
import statsmodels.api as sm 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import decimal

# Read in Training Set

In [2]:
df = pd.read_csv('post_process_train.csv', index_col = 0)
drops = ['GarageArea', 'GarageYrBlt', 'Id', 'FullBath', 'TotRmsAbvGrd', 'X2ndFlrSF', 'BsmtFullBath']
df = df.drop(drops, axis = 1)



In [3]:
len(df.columns)

74

# Read in Kaggle Data

In [4]:
df_kaggle = pd.read_csv('post_process_test.csv', index_col = 0).dropna()
df_kaggle = df_kaggle.drop(drops, axis = 1)


In [5]:
len(df_kaggle.columns)

73

# Process Data

In [6]:
y = df['SalePrice']
df_x = df.drop('SalePrice', axis = 1)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(df_x, y, test_size=0.2, random_state=42)

In [8]:
X_train.shape

(1153, 73)

In [9]:
X_test.shape

(289, 73)

In [10]:
y_train.shape

(1153,)

In [11]:
y_test.shape

(289,)

In [12]:
#process training data
categorical_columns = ['MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour',
                       'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood',
                      'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 
                      'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd',
                       'MasVnrType',
                       'Foundation', 'BsmtFinType1', 'BsmtFinType2',
                       'Heating', 'CentralAir', 'Electrical', 'Functional', 
                       'GarageType', 'GarageFinish', 'PavedDrive', 'Fence',
                       'MiscFeature', 'SaleType', 'SaleCondition']
column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore',sparse = False),categorical_columns))
X_train = column_trans.fit_transform(X_train)
X_test = column_trans.transform(X_test)
df_kaggle = column_trans.transform(df_kaggle)

In [13]:
#process test data
#numerical_columns = ['LotFrontage', 'LotArea', 'SalePrice']
#column_trans = make_column_transformer((OneHotEncoder(handle_unknown='ignore', sparse = False),categorical_columns))


In [14]:
X_train.shape

(1153, 220)

In [15]:
X_test.shape

(289, 220)

In [16]:
df_kaggle.shape

(1447, 220)

# Box Cox Transformation

In [ ]:
df["SalePrice"] = scipy.stats.boxcox(df["SalePrice"], .25)
pd.set_option('display.max_columns', None)
df.head()

##AIC & BIC

In [17]:
X_add_const = sm.add_constant(X_train)
ols = sm.OLS(y_train, X_add_const)
ans = ols.fit()
print(ans.summary())

                            OLS Regression Results                            
Dep. Variable:              SalePrice   R-squared:                       0.805
Model:                            OLS   Adj. R-squared:                  0.769
Method:                 Least Squares   F-statistic:                     22.19
Date:                Fri, 14 Aug 2020   Prob (F-statistic):          1.57e-247
Time:                        23:18:03   Log-Likelihood:                -13602.
No. Observations:                1153   AIC:                         2.757e+04
Df Residuals:                     971   BIC:                         2.849e+04
Df Model:                         181                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.917e+04   5221.096      3.672      0.0

#Multiple Linear Regression

In [34]:
mlr = linear_model.LinearRegression()
mlr


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [33]:
grid_param = [{'copy_X':[True,False], 'fit_intercept': [True,False], 'n_jobs': ['None'], 'normalize':[True,False]}]
para_search = GridSearchCV(estimator=mlr, param_grid=grid_param, scoring='r2', cv=5, return_train_score=True)
para_search = para_search.fit(X_train, y_train)

In [36]:
print(list(para_search.cv_results_.keys()))
para_search.cv_results_
print(para_search.best_score_)
print(para_search.best_params_)
para_search.best_estimator_

['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_copy_X', 'param_fit_intercept', 'param_n_jobs', 'param_normalize', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score', 'split0_train_score', 'split1_train_score', 'split2_train_score', 'split3_train_score', 'split4_train_score', 'mean_train_score', 'std_train_score']
-1.332907326280831e+19
{'copy_X': True, 'fit_intercept': False, 'n_jobs': 'None', 'normalize': True}


LinearRegression(copy_X=True, fit_intercept=False, n_jobs='None',
                 normalize=True)

In [37]:
mlr_best = para_search.best_estimator_
mlr_best.fit(X_train, y_train)
mlr_best.score(X_train, y_train)

0.8053297545506898

In [38]:
mlr_best.score(X_test, y_test)

-7719809123848478.0

In [22]:
lasso = Lasso()
lasso



Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [23]:
alpha_lasso = [1e-15, 1e-10, 1e-8, 1e-5,1e-4, 1e-3,1e-2, 1, 5, 10]

In [24]:
lasso_grid_param = [{'alpha':[alpha_lasso], 'copy_X':[True,False],'fit_intercept': [True,False], 
               'normalize': [True,False], 'positive':[True,False], 'precompute':[True,False],
               'selection':['cyclic', 'random']}]
lasso_para_search = GridSearchCV(estimator=lasso, param_grid=lasso_grid_param, scoring='r2', cv=5, return_train_score=True)
lasso_para_search = para_search.fit(X_train, y_train)

In [25]:
lasso_para_search.cv_results_
print(lasso_para_search.best_score_)
print(lasso_para_search.best_params_)
lasso_para_search.best_estimator_

-1.332907326280831e+19
{'copy_X': True, 'fit_intercept': False, 'n_jobs': 'None', 'normalize': True}


LinearRegression(copy_X=True, fit_intercept=False, n_jobs='None',
                 normalize=True)

In [26]:
lasso_best = lasso_para_search.best_estimator_
lasso_best.fit(X_train, y_train)
lasso_best.score(X_train, y_train)

0.8053297545506898

In [27]:
lasso_best.score(X_test, y_test)

-7719809123848478.0

# Random Forest

In [28]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100)
rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=100, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [29]:
rf.score(X_train, y_train)

0.9538759632828024

In [30]:
rf.score(X_test, y_test)

0.7274191590802601

In [46]:
rf.predict(df_kaggle)

array([137349.59      , 151186.        , 191990.87261905, ...,
       116360.        , 115695.        , 200733.95      ])

# Beginning of XGBoost

In [42]:
mod = xgb.XGBRegressor(
    gamma=1,                 
    learning_rate=0.01,
    max_depth=3,
    n_estimators=10000,                                                                    
    subsample=0.8,
    random_state=34
) 

mod.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=1, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.01, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=10000, n_jobs=0, num_parallel_tree=1,
             objective='reg:squarederror', random_state=34, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [43]:
mod.score(X_train,y_train)

0.9568361742365035

In [44]:
mod.score(X_test,y_test)

0.7358228541407463

In [47]:
mod.predict(df_kaggle)

array([125367.164, 155300.12 , 190781.36 , ..., 117868.12 , 112355.164,
       200231.52 ], dtype=float32)

# SVR

In [39]:
svr = SVR() #epsilon = 1e-4, gamma=1
svr.fit(X_train,y_train)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [40]:
svr.score(X_train,y_train)

-0.047697272717932115

In [41]:
svr.score(X_test,y_test)

-0.010905633953309124

# Undo Box Cox Transformation

In [ ]:
df["SalePrice"] = scipy.special.inv_boxcox(df["SalePrice"], 0.25)
df.head()